In [ ]:
!pip install torchmetrics

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [ ]:
FILL = "FILL"

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2023-12-22 23:23:25--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.28’

smsspamcollection.z     [  <=>               ] 198.65K   436KB/s    in 0.5s    

2023-12-22 23:23:26 (436 KB/s) - ‘smsspamcollection.zip.28’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2023-12-22 23:23:27--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.29’

smsspamcollection.z     [  <=>               ] 198.65K   436KB/s    in 0.5s    

2023-12-22 23:23:28 (436 KB/s) - ‘smsspamcollection.zip.29’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
text = df.text.values
labels = df.label.values

In [ ]:
# Set to the GPT2Tokenizer and set lower case to True
tokenizer =  GPT2Tokenizer.from_pretrained("gpt2") #GPT2 tokenizer; note here we do not use the usual pytorch tokenizer
tokenizer.pad_token = tokenizer.eos_token#FILL #the question here is how we pad the tokenizer
# Set the padding to 'left' or 'right'?
# Remember we want to use the last token's embedding to represent the entire sentence
tokenizer.padding_side = 'left' #we have to use left because we want the emebeddings to represent the sentence.

In [ ]:
labels

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

Hi! You just spoke to MANEESHA V. We'd like to know if you were satisfied with the experience. Reply Toll Free with Yes or No.

╒═════════════╤═════════════╕
│ Tokens      │   Token IDs │
╞═════════════╪═════════════╡
│ Hi          │       17250 │
├─────────────┼─────────────┤
│ !           │           0 │
├─────────────┼─────────────┤
│ ĠYou        │         921 │
├─────────────┼─────────────┤
│ Ġjust       │         655 │
├─────────────┼─────────────┤
│ Ġspoke      │        5158 │
├─────────────┼─────────────┤
│ Ġto         │         284 │
├─────────────┼─────────────┤
│ ĠMAN        │       17254 │
├─────────────┼─────────────┤
│ E           │          36 │
├─────────────┼─────────────┤
│ ES          │        1546 │
├─────────────┼─────────────┤
│ HA          │        7801 │
├─────────────┼─────────────┤
│ ĠV          │         569 │
├─────────────┼─────────────┤
│ .           │          13 │
├─────────────┼─────────────┤
│ ĠWe         │         775 │
├─────────────┼─────────────┤
│ 

In [ ]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  # Use the tokenizer and the encode_plus methods to return the right data we'll need
  # Set max_length = 32 and return_tokens = 'pt'
  # Set other fields to the appropriate booleans needed
  #where are the encode_plus methods; what is particular about this method succintly describe what it does
  #1) the encode_plus method tokenizes and prepares for the model a sequence or  a pair of sequences
  # 'pt': Return PyTorch torch.Tensor objects.
  return tokenizer.encode_plus(text=input_text, return_tensors='pt',padding='max_length', return_attention_mask=True, truncation=True, max_length=32)#FILL



for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


# Gather all the torch_id, attention masks, and labels
token_id = [x.squeeze(0) for x in token_id]
#what are all the token IDs and how do we use them;
attention_masks = [x.squeeze(0) for x in attention_masks] #FILL
#
labels = labels #FILL what is the use of labels what am I expected to do with the labels here

In [ ]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    print(tokens)
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

['<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', 'No', '..', 'its', 'Ġful', 'Ġof', 'Ġsong', 'Ġlyrics', '..', 'Ċ']
╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──────────────────┤
│ <|endoftext|> │       50256 │                0 │
├───────────────┼─────────────┼──

In [ ]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
# Use train_test_split
train_idx, val_idx = train_test_split(np.arange(len(labels)), test_size=val_ratio, shuffle=True, stratify=labels)#FILL

# Train and validation sets
# Set to TensorDataset

train_set = TensorDataset(torch.tensor(train_idx)) #Fill
val_set = TensorDataset(torch.tensor(val_idx)) #Fill

# Prepare DataLoader
train_dataloader = DataLoader(train_set, batch_size=batch_size) #FILL
validation_dataloader = DataLoader(val_set, batch_size=batch_size) #FILL


### Load specific versions of the model

In [ ]:
# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states

config = GPT2Config.from_pretrained('gpt2')#FILL

# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
model = GPT2ForSequenceClassification.from_pretrained('gpt2')#FILL

# Set the pad token id to the eos token id
model.config.pad_token_id = tokenizer.eos_token_id
# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer.eos_token_id

50256

### Set the model to the right device

In [ ]:
# If on GPU, do as below
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')#FILL


In [ ]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [ ]:
# Print all the layers of this GPT2 model and the number of parameters per layer
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = True
# Turn off gradients using the above#
#model.no_grad()
params = torch.cat([param.flatten() for param in model.parameters()]) #they are flattened becaus ethey must have the same dimension in the tensor
#here you have to figure out what flatten does
# flatten()
if not fine_tune:
  for i, param in enumerate(model.parameters()):
      param.requires_grad_(False)

total_parameters = torch.numel(params)
#total_paramters = len(params)
assert(total_parameters == 124441344)

### Train the model

In [ ]:
# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = Accuracy(task='binary').to(device)
recall = Recall(task='binary').to(device)
precision = Precision(task='binary').to(device)
auroc = AUROC(task='binary').to(device)

In [ ]:
for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train(True)

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device

        batch = [x.to(device) for x in batch[0]]
        #print(batch)
        # Unpack the batch
        inputs_tmp = [token_id[int(x.item())] for x in batch]
        attention_tmp = [attention_masks[int(x.item())] for x in batch]
        labels_tmp = torch.tensor([labels[int(x.item())] for x in batch]).to(device)

        inputs_tmp = torch.stack(inputs_tmp, 0).type('torch.LongTensor')
        attention_tmp = torch.stack(attention_tmp, 0).type('torch.FloatTensor')
        #labels_tmp = torch.tensor([torch.tensor(labels[int(x.item())]).type('torch.FloatTensor').to(device) for x in batch]).type('torch.FloatTensor').to(device)
        #labels_tmp = torch.cat(labels_tmp, dim=1).type('torch.FloatTensor').to(device)

        #print(attention_tmp)
        #print(inputstmp)
        #print(labels_tmp)
        #how do you unpack the batch and why does it have to be unpacked

        # Set gradients to zero
        model.zero_grad() #FILL

        # Forward pass
        train_output = model(input_ids=inputs_tmp.to(device), attention_mask=attention_tmp.to(device), labels=labels_tmp, token_type_ids = None)

        # Backward pass# Main training / validation loop

        loss = train_output.loss
        loss.backward() #FILL
        optimizer.step()#FILL



        # Update tracking variables
        tr_loss +=  loss.item()
        nb_tr_examples += 1#ILL
        nb_tr_steps += 1# FILL

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()#FILL

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch =  [x.to(device) for x in batch[0]]#FILL

        # Unpack the batch
        #FILL
        inputs_tmp = [token_id[int(x.item())] for x in batch]
        attention_tmp = [attention_masks[int(x.item())]for x in batch]
        labels_tmp = torch.tensor([labels[int(x.item())] for x in batch]).to(device)

        inputs_tmp = torch.stack(inputs_tmp, 0).type('torch.LongTensor')
        attention_tmp = torch.stack(attention_tmp, 0).type('torch.FloatTensor')

        with torch.no_grad():
          # Forward pass
            eval_output = model(input_ids=inputs_tmp.to(device), token_type_ids = None, attention_mask=attention_tmp.to(device))

        # Calculate validation metrics
        labels2 = labels_tmp#FILL
        predicted_labels = eval_output.logits.argmax(axis=-1).flatten().to(device)#FILL

        val_accuracy.append(accuracy(predicted_labels, labels2))
        val_recall.append(recall(predicted_labels, labels2))
        val_precision.append(precision(predicted_labels, labels2))
        val_auroc.append(auroc(predicted_labels, labels2))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
Epoch:  50%|█████     | 1/2 [00:40<00:40, 40.11s/it]


	 - Train loss: 0.1711
	 - Validation Accuracy: 0.9759
	 - Validation Precision: 0.8000
	 - Validation Recall: 0.8861
	 - Validation AUROC: 0.8831



Epoch: 100%|██████████| 2/2 [01:18<00:00, 39.38s/it]


	 - Train loss: 0.0450
	 - Validation Accuracy: 0.9911
	 - Validation Precision: 0.8702
	 - Validation Recall: 0.8872
	 - Validation AUROC: 0.8911



### Test on a specific sentence, see the outcome

In [ ]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

#here

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Spam


### Questions

Question 1: Run the above by fine tuning GPT2 and the classfier head and by not doing this (using GPT2 as a feature encoder). What is the gap between this? What are the metrics we get in each case?

Solution: FILL Paste your output here


This is when fine_tuning is set ti true:

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
warnings.warn(*args, **kwargs)  # noqa: B028

Epoch:  50%|█████     | 1/2 [00:41<00:41, 41.05s/it]
> Train loss: 0.1206
> Validation Accuracy: 0.9866

> Validation Precision: 0.8893

> Validation Recall: 0.9233

> Validation AUROC: 0.9279


Epoch: 100%|██████████| 2/2 [01:19<00:00, 39.73s/it]

> Train loss: 0.0396

> Validation Accuracy: 0.9830

> Validation Precision: 0.8721

> Validation Recall: 0.9281

> Validation AUROC: 0.9275


By contrast here is the out put without fine_tuning:


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028

Epoch:  50%|█████     | 1/2 [00:29<00:29, 29.65s/it]

> Train loss: 3.2346

> Validation Accuracy: 0.2410
> Validation Precision: 0.1436
> Validation Recall: 0.8417
> Validation AUROC: 0.4780

Epoch: 100%|██████████| 2/2 [01:00<00:00, 30.15s/it]
> Train loss: 3.2121

> Validation Accuracy: 0.2410

> Validation Precision: 0.1436

> Validation Recall: 0.8417

> Validation AUROC: 0.4780


As seen above, the loss is significantly greater and all 4 key performance indicators are lower in comparison. Only recall remains more or less closer.